In [1]:
import os
import time
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
isbns = pd.read_json(os.path.join('data','sledovat.json'), dtype={'ISBN':str})['ISBN'].drop_duplicates().to_list()
isbns = [i for i in isbns if i != 'None']
print(f"Položek ke stažení: {len(isbns)}")

Položek ke stažení: 2829


In [3]:
isbns

['9788027904853',
 '9788027913152',
 '9788020615770',
 '9788027084265',
 '9788027913558',
 '9788072444748',
 '9788072444922',
 '9788075255396',
 '9788076374591',
 '9788024289472',
 '9788074704758',
 '9788088367291',
 '9788072157020',
 '9788072529704',
 '9788090837690',
 '9788072116249',
 '9788075884374',
 '9788088067757',
 '9788088406501',
 '9788024952901',
 '9788024287539',
 '9788072444816',
 '9788072225309',
 '9788011026769',
 '9788027801145',
 '9788081116148',
 '9788076454248',
 '9788027912919',
 '9788027912926',
 '9788076617254',
 '9788026447375',
 '9788024288826',
 '9788088367307',
 '9788075884497',
 '9788075512475',
 '9788075941336',
 '9788020461773',
 '9788076017481',
 '9781847498885',
 '9788088467649',
 '9788075884619',
 '9788072278893',
 '9788072157181',
 '9788025740910',
 '9788072872770',
 '9788075464613',
 '9788027721405',
 '9788076900387',
 '9788024651682',
 '9788090812772',
 '9788090812765',
 '9788076910058',
 '9788026446965',
 '9788087469606',
 '9788088318354',
 '97880761

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'}
def scrape_goodreads(isbn):
    
    kniha = {
        'ISBN': isbn,
        'GR_date': datetime.datetime.now().replace(microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
    }
    
    try:
        r = requests.get(f"""https://www.goodreads.com/search?q={isbn}&search_type=isbn&search%5Bfield%5D=isbn""", timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
    except Exception as e:
            print(e)
            try:
                time.sleep(120)
                r = requests.get(f"""https://www.goodreads.com/search?q={isbn}&search_type=isbn&search%5Bfield%5D=isbn""", timeout=10, headers=headers)
                soup = BeautifulSoup(r.text, "html.parser")
            except:
                return {}
        
    try:
        kniha['GR_title'] = soup.find('title').text.split('|')[0].strip()
    except:
        return kniha
    
    if ('Search results for ' in kniha['GR_title']) or ('request took too long' in kniha['GR_title']):
        kniha['GR_title'] = None
        return kniha
        
    try:
        kniha['GR_rating'] = float(soup.find(class_='RatingStatistics__rating').text.strip())
    except:
        pass
    try:
        kniha['GR_ratings_count'] = int(soup.find('span', {'data-testid': 'ratingsCount'}).text.split('\xa0')[0].strip().replace(",",""))
    except:
        pass
    try:
        kniha['GR_reviews'] = int(soup.find('span', {'data-testid': 'reviewsCount'}).text.split('\xa0')[0].strip().replace(",",""))
    except:
        pass
    try:
        kniha['GR_published'] = soup.find('p', {'data-testid': 'publicationInfo'}).text.split('lished')[1].strip()
    except:
        pass
    return kniha

In [5]:
scrape_goodreads("9788076111226")

{'ISBN': '9788076111226',
 'GR_date': '2024-04-02 10:22:16',
 'GR_title': 'Gazely by Patrik Hartl',
 'GR_rating': 3.75,
 'GR_ratings_count': 338,
 'GR_reviews': 21,
 'GR_published': 'October 13, 2023'}

In [6]:
current_date = datetime.datetime.now()
date_string = current_date.strftime("%Y_%m_%d")
print(date_string)

2024_04_02


In [7]:
if not os.path.exists(f'data_raw/goodreads/{date_string}'):
    os.makedirs(f'data_raw/goodreads/{date_string}')

In [8]:
greads = []
count = 0
for i in isbns:
    count += 1
    greads.append(scrape_goodreads(i))
    if count % 20 == 0:
        pd.DataFrame(greads).to_json(os.path.join(f'data_raw/goodreads/{date_string}',f'goodreads_{date_string}_{(int(count/20)):04d}.json'))
        print(f'goodreads_{date_string}_{(int(count/20)):04d}.json')
        greads = []

goodreads_2024_04_02_0001.json
goodreads_2024_04_02_0002.json
goodreads_2024_04_02_0003.json
goodreads_2024_04_02_0004.json
goodreads_2024_04_02_0005.json
goodreads_2024_04_02_0006.json
goodreads_2024_04_02_0007.json
goodreads_2024_04_02_0008.json
goodreads_2024_04_02_0009.json
goodreads_2024_04_02_0010.json
goodreads_2024_04_02_0011.json
goodreads_2024_04_02_0012.json
goodreads_2024_04_02_0013.json
goodreads_2024_04_02_0014.json
goodreads_2024_04_02_0015.json
goodreads_2024_04_02_0016.json
goodreads_2024_04_02_0017.json
goodreads_2024_04_02_0018.json
goodreads_2024_04_02_0019.json
goodreads_2024_04_02_0020.json
goodreads_2024_04_02_0021.json
goodreads_2024_04_02_0022.json
goodreads_2024_04_02_0023.json
goodreads_2024_04_02_0024.json
goodreads_2024_04_02_0025.json
goodreads_2024_04_02_0026.json
HTTPSConnectionPool(host='www.goodreads.com', port=443): Read timed out. (read timeout=10)
goodreads_2024_04_02_0027.json
goodreads_2024_04_02_0028.json
goodreads_2024_04_02_0029.json
goodreads_